In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Assuming we have our cleaned data from previous analysis
# df_ml_clean should be available from the previous notebook

print("🔍 DETAILED ANALYSIS: PLANNERS & PARTS")
print("=" * 50)

🔍 DETAILED ANALYSIS: PLANNERS & PARTS


In [5]:
file_path = '/home/alaskapycoder/DataScience/MAY_CYCLE_COUNT.xlsx'
sheet_name = 'WestFargo'

In [9]:
try:
    # Load raw data
    df_raw = pd.read_excel(file_path, sheet_name=sheet_name)
    print(f"✅ Loaded {df_raw.shape[0]} rows from {sheet_name}")
    
    # Clean column names
    df_raw.columns = df_raw.columns.str.strip()
    print(f"📋 Columns: {list(df_raw.columns)}")
    
except Exception as e:
    print(f"❌ Error loading file: {e}")
    print("Please ensure MAY_CYCLE_COUNT.xlsx is in the current directory")

# %%
# Clean and prepare data for analysis
print("\n🧹 CLEANING AND PREPARING DATA")
print("=" * 35)

# Remove unwanted columns (as specified in original request)
columns_to_drop = ['NOTES', 'REASON FOR VARIANCE']
df_clean = df_raw.drop(columns=[col for col in columns_to_drop if col in df_raw.columns], errors='ignore')

# Remove rows with missing critical data
df_clean = df_clean.dropna(subset=['VARIANCE', 'PLANNER', 'Product number']).copy()


✅ Loaded 572 rows from WestFargo
📋 Columns: ['DATE', 'Product number', 'DESCRIPTION', 'LOCATION', 'START', 'END', 'DIFFERENCE', 'PLANNER', 'VARIANCE', 'NOTES', 'REASON FOR VARIANCE']

🧹 CLEANING AND PREPARING DATA


In [10]:
columns_to_drop = ['NOTES', 'REASON FOR VARIANCE']
df_clean = df_raw.drop(columns=[col for col in columns_to_drop if col in df_raw.columns], errors='ignore')

# Remove rows with missing critical data
df_clean = df_clean.dropna(subset=['VARIANCE', 'PLANNER', 'Product number']).copy()

# Create key calculated fields
df_clean['ABS_VARIANCE'] = abs(df_clean['VARIANCE'])
df_clean['ABS_DIFFERENCE'] = abs(df_clean['DIFFERENCE'])


In [11]:
def categorize_variance(variance):
    abs_var = abs(variance)
    if abs_var <= 0.05:  # 5% threshold
        return 'LOW'
    elif abs_var <= 0.15:  # 15% threshold
        return 'MEDIUM'
    else:
        return 'HIGH'

df_clean['VARIANCE_CATEGORY'] = df_clean['VARIANCE'].apply(categorize_variance)

# Clean planner names (remove extra spaces)
df_clean['PLANNER'] = df_clean['PLANNER'].str.strip()
df_clean['Product number'] = df_clean['Product number'].str.strip()

print(f"✅ Data cleaned and prepared")
print(f"📊 Final dataset shape: {df_clean.shape}")
print(f"🎯 Variance categories: {df_clean['VARIANCE_CATEGORY'].value_counts()}")

# Save this as our working dataframe
df_analysis = df_clean.copy()

# %%
# Quick data validation
print("\n🔍 DATA VALIDATION")
print("=" * 20)

print("Key Statistics:")
print(f"  • Total items: {len(df_analysis)}")
print(f"  • Unique planners: {df_analysis['PLANNER'].nunique()}")
print(f"  • Unique products: {df_analysis['Product number'].nunique()}")
print(f"  • Date range: {df_analysis['DATE'].min()} to {df_analysis['DATE'].max()}")

print(f"\nVariance Distribution:")
for category in ['LOW', 'MEDIUM', 'HIGH']:
    count = (df_analysis['VARIANCE_CATEGORY'] == category).sum()
    pct = count / len(df_analysis) * 100
    print(f"  • {category}: {count} items ({pct:.1f}%)")

✅ Data cleaned and prepared
📊 Final dataset shape: (255, 12)
🎯 Variance categories: VARIANCE_CATEGORY
LOW       162
HIGH       66
MEDIUM     27
Name: count, dtype: int64

🔍 DATA VALIDATION
Key Statistics:
  • Total items: 255
  • Unique planners: 10
  • Unique products: 186
  • Date range: 2025-05-15 00:00:00 to 2025-05-29 00:00:00

Variance Distribution:
  • LOW: 162 items (63.5%)
  • MEDIUM: 27 items (10.6%)
  • HIGH: 66 items (25.9%)


In [12]:
planner_stats = df_analysis.groupby('PLANNER').agg({
    'Product number': 'count',  # Total items managed
    'ABS_VARIANCE': ['mean', 'std', 'max'],  # Variance statistics
    'DIFFERENCE': ['mean', 'std'],  # Difference statistics
    'START': ['mean', 'sum'],  # Inventory value managed
}).round(4)

# Flatten column names
planner_stats.columns = ['Total_Items', 'Avg_Abs_Variance', 'Std_Variance', 'Max_Variance',
                        'Avg_Difference', 'Std_Difference', 'Avg_Inventory_Value', 'Total_Inventory_Value']

# Add high variance count and rate
high_variance_counts = df_analysis.groupby('PLANNER')['VARIANCE_CATEGORY'].apply(
    lambda x: (x == 'HIGH').sum()
).rename('High_Variance_Count')

planner_stats = planner_stats.join(high_variance_counts)
planner_stats['High_Variance_Rate'] = (planner_stats['High_Variance_Count'] / 
                                      planner_stats['Total_Items'] * 100).round(1)

# Calculate overall risk score
planner_stats['Risk_Score'] = (planner_stats['Avg_Abs_Variance'] * 100 + 
                              planner_stats['High_Variance_Rate']).round(1)

# Sort by risk score
planner_stats = planner_stats.sort_values('Risk_Score', ascending=False)

print("PLANNER PERFORMANCE SUMMARY:")
print("=" * 30)
display_cols = ['Total_Items', 'High_Variance_Count', 'High_Variance_Rate', 'Avg_Abs_Variance', 'Risk_Score']
print(planner_stats[display_cols])

# %%
# Categorize planners by risk level
print("\n🚨 PLANNER RISK CATEGORIES")
print("=" * 30)


PLANNER PERFORMANCE SUMMARY:
         Total_Items  High_Variance_Count  High_Variance_Rate  \
PLANNER                                                         
01BIFA             1                    1               100.0   
01LTEA            15                    5                33.3   
01TG1A            41                   19                46.3   
01LAMA            16                   10                62.5   
01FVWA             5                    3                60.0   
01SPRA            12                    4                33.3   
01HDWA            77                   21                27.3   
01PRSA             4                    1                25.0   
01TRKA            72                    2                 2.8   
01SHPA             8                    0                 0.0   

         Avg_Abs_Variance  Risk_Score  
PLANNER                                
01BIFA             2.0000       300.0  
01LTEA             2.3081       264.1  
01TG1A             0.8924     

In [13]:
high_risk_threshold = planner_stats['Risk_Score'].quantile(0.75)
medium_risk_threshold = planner_stats['Risk_Score'].quantile(0.5)

high_risk_planners = []
medium_risk_planners = []
low_risk_planners = []

for planner, row in planner_stats.iterrows():
    if row['Risk_Score'] > high_risk_threshold:
        risk_level = "🔴 HIGH RISK"
        high_risk_planners.append(planner)
    elif row['Risk_Score'] > medium_risk_threshold:
        risk_level = "🟡 MEDIUM RISK" 
        medium_risk_planners.append(planner)
    else:
        risk_level = "🟢 LOW RISK"
        low_risk_planners.append(planner)
    
    print(f"\n{planner} - {risk_level}")
    print(f"  📊 Manages {int(row['Total_Items'])} items")
    print(f"  📈 High variance rate: {row['High_Variance_Rate']:.1f}%")
    print(f"  📉 Average variance: {row['Avg_Abs_Variance']:.1%}")
    print(f"  ⚡ Risk score: {row['Risk_Score']:.1f}")



01BIFA - 🔴 HIGH RISK
  📊 Manages 1 items
  📈 High variance rate: 100.0%
  📉 Average variance: 200.0%
  ⚡ Risk score: 300.0

01LTEA - 🔴 HIGH RISK
  📊 Manages 15 items
  📈 High variance rate: 33.3%
  📉 Average variance: 230.8%
  ⚡ Risk score: 264.1

01TG1A - 🔴 HIGH RISK
  📊 Manages 41 items
  📈 High variance rate: 46.3%
  📉 Average variance: 89.2%
  ⚡ Risk score: 135.5

01LAMA - 🟡 MEDIUM RISK
  📊 Manages 16 items
  📈 High variance rate: 62.5%
  📉 Average variance: 38.5%
  ⚡ Risk score: 101.0

01FVWA - 🟡 MEDIUM RISK
  📊 Manages 5 items
  📈 High variance rate: 60.0%
  📉 Average variance: 37.0%
  ⚡ Risk score: 97.0

01SPRA - 🟢 LOW RISK
  📊 Manages 12 items
  📈 High variance rate: 33.3%
  📉 Average variance: 31.4%
  ⚡ Risk score: 64.7

01HDWA - 🟢 LOW RISK
  📊 Manages 77 items
  📈 High variance rate: 27.3%
  📉 Average variance: 33.9%
  ⚡ Risk score: 61.2

01PRSA - 🟢 LOW RISK
  📊 Manages 4 items
  📈 High variance rate: 25.0%
  📉 Average variance: 35.0%
  ⚡ Risk score: 60.0

01TRKA - 🟢 LOW RIS

In [14]:
print("\n🎯 ITEMS REQUIRING IMMEDIATE ATTENTION")
print("=" * 45)

# Get high priority items (HIGH variance)
high_priority = df_analysis[df_analysis['VARIANCE_CATEGORY'] == 'HIGH'].copy()
high_priority = high_priority.sort_values('ABS_VARIANCE', ascending=False)

print(f"🔴 HIGH PRIORITY - RECOUNT REQUIRED ({len(high_priority)} items)")
print("=" * 60)

for idx, (_, item) in enumerate(high_priority.iterrows()):
    print(f"\n{idx+1:2d}. PART: {item['Product number']}")
    print(f"     DESCRIPTION: {item['DESCRIPTION']}")
    print(f"     PLANNER: {item['PLANNER']}")
    print(f"     EXPECTED: {int(item['START']):,} | FOUND: {int(item['END']):,} | DIFF: {int(item['DIFFERENCE']):,}")
    print(f"     VARIANCE: {item['VARIANCE']:.1%} | ACTION: IMMEDIATE RECOUNT")

# %%
# Get medium priority items
medium_priority = df_analysis[df_analysis['VARIANCE_CATEGORY'] == 'MEDIUM'].copy()
medium_priority = medium_priority.sort_values('ABS_VARIANCE', ascending=False)

print(f"\n🟡 MEDIUM PRIORITY - REVIEW RECOMMENDED ({len(medium_priority)} items)")
print("=" * 65)

for idx, (_, item) in enumerate(medium_priority.head(10).iterrows()):  # Show top 10
    print(f"\n{idx+1:2d}. PART: {item['Product number']}")
    print(f"     DESCRIPTION: {item['DESCRIPTION']}")
    print(f"     PLANNER: {item['PLANNER']}")
    print(f"     EXPECTED: {int(item['START']):,} | FOUND: {int(item['END']):,} | DIFF: {int(item['DIFFERENCE']):,}")
    print(f"     VARIANCE: {item['VARIANCE']:.1%} | ACTION: SPOT CHECK")

if len(medium_priority) > 10:
    print(f"\n... and {len(medium_priority) - 10} more medium priority items")



🎯 ITEMS REQUIRING IMMEDIATE ATTENTION
🔴 HIGH PRIORITY - RECOUNT REQUIRED (66 items)

 1. PART: EVWS112
     DESCRIPTION: VINYL WINDOW SCREW 1 1/2IN    
     PLANNER: 01LTEA
     EXPECTED: 152,950 | FOUND: 4,750,000 | DIFF: 4,597,050
     VARIANCE: 3005.6% | ACTION: IMMEDIATE RECOUNT

 2. PART: TG084L194
     DESCRIPTION: 16FT 2IN 2TG BOTTOM EXTRUSION 
     PLANNER: 01TG1A
     EXPECTED: -16 | FOUND: 80 | DIFF: 96
     VARIANCE: -600.0% | ACTION: IMMEDIATE RECOUNT

 3. PART: TG018L146
     DESCRIPTION: 12FT 2IN 2TG TOP EXTRUSION    
     PLANNER: 01TG1A
     EXPECTED: -37 | FOUND: 163 | DIFF: 200
     VARIANCE: -540.5% | ACTION: IMMEDIATE RECOUNT

 4. PART: BBAM18W
     DESCRIPTION: 18-W ANGLE MT LEG OUT BAG 18HI
     PLANNER: 01HDWA
     EXPECTED: 5 | FOUND: 32 | DIFF: 27
     VARIANCE: 540.0% | ACTION: IMMEDIATE RECOUNT

 5. PART: BBAM16S
     DESCRIPTION: 16-S ANGLE MT LEG IN BAG 16 HI
     PLANNER: 01HDWA
     EXPECTED: -1 | FOUND: 4 | DIFF: 5
     VARIANCE: -500.0% | ACTION: IMMED

In [15]:
today = datetime.now()
schedule_items = []

# Schedule HIGH priority items (spread over 3 days)
for idx, (_, item) in enumerate(high_priority.iterrows()):
    schedule_date = today + timedelta(days=(idx % 3) + 1)
    schedule_items.append({
        'Part_Number': item['Product number'],
        'Description': item['DESCRIPTION'],
        'Planner': item['PLANNER'],
        'Current_Qty': int(item['END']),
        'Expected_Qty': int(item['START']),
        'Variance_Pct': f"{item['VARIANCE']:.1%}",
        'Priority': 'HIGH',
        'Suggested_Date': schedule_date.strftime('%Y-%m-%d'),
        'Day_of_Week': schedule_date.strftime('%A'),
        'Action': 'FULL_RECOUNT',
        'Estimated_Time_Min': 30,
        'Notes': 'Immediate attention required'
    })

# Schedule MEDIUM priority items (spread over week 2)
for idx, (_, item) in enumerate(medium_priority.iterrows()):
    schedule_date = today + timedelta(days=7 + (idx % 7))
    schedule_items.append({
        'Part_Number': item['Product number'],
        'Description': item['DESCRIPTION'],
        'Planner': item['PLANNER'],
        'Current_Qty': int(item['END']),
        'Expected_Qty': int(item['START']),
        'Variance_Pct': f"{item['VARIANCE']:.1%}",
        'Priority': 'MEDIUM',
        'Suggested_Date': schedule_date.strftime('%Y-%m-%d'),
        'Day_of_Week': schedule_date.strftime('%A'),
        'Action': 'SPOT_CHECK',
        'Estimated_Time_Min': 15,
        'Notes': 'Review and verify count'
    })

In [16]:
schedule_df = pd.DataFrame(schedule_items)

# Export to CSV
filename = f"WestFargo_CycleCount_Schedule_{today.strftime('%Y%m%d')}.csv"
schedule_df.to_csv(filename, index=False)

print(f"✅ ERP scheduling file created: {filename}")
print(f"📊 Total items scheduled: {len(schedule_df)}")

# Show preview
print(f"\nPreview of scheduling file:")
print(schedule_df[['Part_Number', 'Planner', 'Priority', 'Suggested_Date', 'Action']].head(10))

# %%
# Workload summary by planner
print(f"\n👥 WORKLOAD DISTRIBUTION")
print("=" * 25)

workload_summary = schedule_df.groupby(['Planner', 'Priority']).agg({
    'Part_Number': 'count',
    'Estimated_Time_Min': 'sum'
}).round(0)

workload_summary.columns = ['Item_Count', 'Total_Minutes']

# Add total time in hours
workload_pivot = schedule_df.groupby('Planner').agg({
    'Part_Number': 'count',
    'Estimated_Time_Min': 'sum'
}).round(0)
workload_pivot['Total_Hours'] = (workload_pivot['Estimated_Time_Min'] / 60).round(1)
workload_pivot.columns = ['Total_Items', 'Total_Minutes', 'Total_Hours']

print("Workload by Planner:")
print(workload_pivot.sort_values('Total_Hours', ascending=False))

# %% [markdown]
# ## 5. Action Plans by Planner

# %%
print("\n📋 PLANNER-SPECIFIC ACTION PLANS")
print("=" * 35)

for planner in planner_stats.index:
    planner_data = planner_stats.loc[planner]
    planner_items = df_analysis[df_analysis['PLANNER'] == planner]
    
    print(f"\n{'='*50}")
    print(f"📝 ACTION PLAN: {planner}")
    print(f"{'='*50}")
    
    # Risk assessment and recommendations
    if planner in high_risk_planners:
        print("🔴 STATUS: HIGH RISK")
        print("🚨 IMMEDIATE ACTIONS REQUIRED:")
        print("   • Schedule training session within 48 hours")
        print("   • Implement daily supervisor checks")
        print("   • Review counting procedures step-by-step")
        print("   • Consider reassigning complex items temporarily")
        
    elif planner in medium_risk_planners:
        print("🟡 STATUS: MEDIUM RISK")
        print("📚 RECOMMENDED ACTIONS:")
        print("   • Provide refresher training this week")
        print("   • Weekly check-ins for next month")
        print("   • Review most problematic items together")
        
    else:
        print("🟢 STATUS: PERFORMING WELL")
        print("🌟 RECOGNITION:")
        print("   • Acknowledge excellent performance")
        print("   • Consider as mentor for other planners")
        print("   • May handle more complex items")
    
    # Specific problem items for this planner
    planner_high_items = planner_items[planner_items['VARIANCE_CATEGORY'] == 'HIGH']
    planner_medium_items = planner_items[planner_items['VARIANCE_CATEGORY'] == 'MEDIUM']
    
    if len(planner_high_items) > 0:
        print(f"\n🎯 HIGH PRIORITY ITEMS TO FOCUS ON:")
        for _, item in planner_high_items.head(5).iterrows():
            print(f"   • {item['Product number']}: {item['VARIANCE']:.1%} variance")
    
    if len(planner_medium_items) > 0:
        print(f"\n📋 MEDIUM PRIORITY ITEMS TO REVIEW:")
        for _, item in planner_medium_items.head(3).iterrows():
            print(f"   • {item['Product number']}: {item['VARIANCE']:.1%} variance")
    
    # Performance metrics
    print(f"\n📊 CURRENT PERFORMANCE:")
    print(f"   • Items managed: {int(planner_data['Total_Items'])}")
    print(f"   • High variance items: {int(planner_data['High_Variance_Count'])}")
    print(f"   • High variance rate: {planner_data['High_Variance_Rate']:.1f}%")
    print(f"   • Average variance: {planner_data['Avg_Abs_Variance']:.1%}")
    print(f"   • Risk score: {planner_data['Risk_Score']:.1f}")

# %% [markdown]
# ## 6. Executive Summary Export

# %%
print(f"\n📊 CREATING EXECUTIVE SUMMARY")
print("=" * 30)

# Create executive summary metrics
exec_summary = {
    'Analysis_Date': today.strftime('%Y-%m-%d'),
    'Location': 'WestFargo', 
    'Month_Analyzed': 'May 2025',
    'Total_Items_Analyzed': len(df_analysis),
    'High_Priority_Items': len(high_priority),
    'Medium_Priority_Items': len(medium_priority),
    'Low_Priority_Items': len(df_analysis[df_analysis['VARIANCE_CATEGORY'] == 'LOW']),
    'High_Risk_Planners': len(high_risk_planners),
    'Medium_Risk_Planners': len(medium_risk_planners),
    'Low_Risk_Planners': len(low_risk_planners),
    'Estimated_Recount_Hours': len(high_priority) * 0.5 + len(medium_priority) * 0.25,
    'Potential_Time_Savings_Pct': f"{len(df_analysis[df_analysis['VARIANCE_CATEGORY'] == 'LOW']) / len(df_analysis) * 100:.0f}%",
    'Overall_High_Variance_Rate': f"{len(high_priority) / len(df_analysis) * 100:.1f}%"
}

# Save executive summary
exec_df = pd.DataFrame([exec_summary])
exec_filename = f"WestFargo_Executive_Summary_{today.strftime('%Y%m%d')}.csv"
exec_df.to_csv(exec_filename, index=False)

print(f"✅ Executive summary saved: {exec_filename}")

print(f"\n📋 KEY METRICS:")
for key, value in exec_summary.items():
    if key != 'Analysis_Date':
        print(f"   • {key.replace('_', ' ')}: {value}")

# %%
print(f"\n🎉 ANALYSIS COMPLETE!")
print("=" * 25)

print(f"📁 FILES CREATED:")
print(f"   • {filename} (ERP scheduling)")
print(f"   • {exec_filename} (Executive summary)")

print(f"\n📈 NEXT STEPS:")
print("   1. Import scheduling file into ERP system")
print("   2. Meet with high-risk planners this week")
print("   3. Begin recounting high priority items")
print("   4. Re-run analysis next month with June data")

print(f"\n💡 KEY INSIGHT:")
high_variance_pct = len(high_priority) / len(df_analysis) * 100
low_variance_pct = len(df_analysis[df_analysis['VARIANCE_CATEGORY'] == 'LOW']) / len(df_analysis) * 100
print(f"   • {high_variance_pct:.0f}% of items need immediate attention")
print(f"   • {low_variance_pct:.0f}% of items are counting accurately (save time here!)")
print(f"   • Focus training on {len(high_risk_planners)} high-risk planners")

# %%
# Optional: Save the cleaned dataframe for future use
df_analysis.to_csv(f"WestFargo_CleanData_{today.strftime('%Y%m%d')}.csv", index=False)
print(f"\n💾 Bonus: Clean dataset saved as WestFargo_CleanData_{today.strftime('%Y%m%d')}.csv")
print("   (Use this for future analysis or comparison)")

✅ ERP scheduling file created: WestFargo_CycleCount_Schedule_20250603.csv
📊 Total items scheduled: 93

Preview of scheduling file:
  Part_Number Planner Priority Suggested_Date        Action
0     EVWS112  01LTEA     HIGH     2025-06-04  FULL_RECOUNT
1   TG084L194  01TG1A     HIGH     2025-06-05  FULL_RECOUNT
2   TG018L146  01TG1A     HIGH     2025-06-06  FULL_RECOUNT
3     BBAM18W  01HDWA     HIGH     2025-06-04  FULL_RECOUNT
4     BBAM16S  01HDWA     HIGH     2025-06-05  FULL_RECOUNT
5         NaN  01LTEA     HIGH     2025-06-06  FULL_RECOUNT
6   TG084L194  01TG1A     HIGH     2025-06-04  FULL_RECOUNT
7         NaN  01HDWA     HIGH     2025-06-05  FULL_RECOUNT
8   TG018L194  01TG1A     HIGH     2025-06-06  FULL_RECOUNT
9     LM312HM  01BIFA     HIGH     2025-06-04  FULL_RECOUNT

👥 WORKLOAD DISTRIBUTION
Workload by Planner:
         Total_Items  Total_Minutes  Total_Hours
Planner                                         
01HDWA            32            810         13.5
01TG1A          